In [1]:

import pandas as pd
from wordcloud import WordCloud


In [2]:
txt_file = '먹거리_식품_안전(2017~2012).txt'
# convert txt file to .csv file, and save it
# we expect 20xxxxxx source text
# splits with \t
import re
import csv
import os
from tqdm import tqdm

def allowed_characters(string: str) -> str:
    # returns only allowed characters and convert else to space
    # use regex
    return re.sub(r'[^가-힣0-9a-zA-Z]', ' ', string)

skipped_line_idxs = set()
if not os.path.exists('result.csv'):
    print("processing txt file to csv file")
    with open(txt_file, 'r', encoding='utf-8') as txtfile:
        total_lines = sum(1 for line in txtfile)
    with open('result.csv', 'w', encoding='utf-8', newline='') as csv_output:
        csv_writer = csv.writer(csv_output)
        with open(txt_file, 'r', encoding='utf-8') as txtfile:
            max_lines = -1 # -1 means all lines
            cur_line = 0
            for line in tqdm(txtfile, total=total_lines):
                cur_line += 1
                splits = line.split('\t')
                if len(splits) != 4:
                    skipped_line_idxs.add(cur_line)
                    continue
                # remove \n
                splits[-1] = splits[-1].replace('\n', '')
                # remove special characters
                splits = [allowed_characters(s) for s in splits]
                # write to csv
                csv_writer.writerow(splits)
                if cur_line == max_lines:
                    break
    with open('skipped_line_idxs.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join((str(x) for x in skipped_line_idxs)))



In [3]:
# test csv file with index 3
with open('result.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    lines = 3
    for rows in reader:
        try:
            rows[3]
        except IndexError:
            print(lines)
        lines-=1
        if lines == 0:
            break


In [4]:
import treform as ptm
from sklearn.feature_extraction.text import CountVectorizer
from ExampleManager import PathManager

log_path: None
error_log_path: None
log level: INFO


In [5]:
import sys
csv.field_size_limit(1<<21)
# this is required to read very large csv

131072

In [6]:
import os
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-16.0.2'
os.environ['GIT_PYTHON_GIT_EXECUTABLE'] = r'C:\Program Files\Git\bin\git.exe'
import git

In [7]:
# test packages
from ExampleManager import PathManager
str(PathManager('../stopwords/stopwordsKor.txt'))

C:\Users\박상현\AppData\Roaming\Python\Python310\site-packages\treform\stopwords
Created temporary path C:\tmp\clone\122312
Removed temporary path C:\tmp\clone\122312
parsing repo path


'C:\\Users\\박상현\\AppData\\Roaming\\Python\\Python310\\site-packages\\treform\\stopwords\\stopwordsKor.txt'

In [8]:
corpus = ptm.CorpusFromCSVFile('./result.csv', 3)
pipeline = ptm.Pipeline(ptm.splitter.NLTK(),
                        ptm.tokenizer.Word(),
                        ptm.helper.StopwordFilter(file=str(PathManager('../stopwords/stopwordsKor.txt')))
                        )

parsing repo path


In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\박상현\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [ ]:
result = pipeline.processCorpus(tqdm(corpus))

 54%|█████▎    | 30973/57718 [11:19:50<01:04, 416.63it/s]     

In [ ]:
assert False, "stop here"

In [ ]:
documents = []
for doc in result:
    for sent in doc:
        sentence = ' '.join(sent)
        sentence = re.sub('[^A-Za-z0-9가-힣_ ]+', '', sentence)
        sentence = sentence.strip()
        if len(sentence) > 0:
            documents.append(sentence)

print(len(documents))

In [ ]:
cv = CountVectorizer()
cv_fit = cv.fit_transform(documents)
word_list = cv.get_feature_names()
count_list = cv_fit.toarray().sum(axis=0)
word_hist = dict(zip(word_list, count_list))
words = []
for word, freq in word_hist.items() :
    for i in range(freq) :
        words.append(word)
co = ptm.cooccurrence.CooccurrenceManager()
co_result, vocab = co.calculateCooccurrence(words)
print(str(co_result))
print(str(vocab))
    ########################################################################################################################

In [ ]:
graph_builder = ptm.graphml.GraphMLCreator()
# mode is either with_threshold or without_threshod
mode = 'with_threshold'
if mode is 'without_threshold':
    graph_builder.createGraphML(co_result, vocab, "test1.graphml")
elif mode is 'with_threshold':
    graph_builder.createGraphMLWithThreshold(co_result, word_hist, vocab, "test.graphml", threshold=35.0)
    # 이 경우에만 그림을 그려보도록 하자
    display_limit = 50
    graph_builder.summarize_centrality(limit=display_limit)
    title = '동시출현 기반 그래프'
    file_name = 'test.png'
    graph_builder.plot_graph(title, file=file_name)


In [ ]:
splits

In [ ]:

# Create a Pandas dataframe
df = pd.DataFrame({'text': ['apple banana apple cherry cherry date']})

# Generate a word cloud image
wordcloud = WordCloud().generate(' '.join(df['text']))

# Display the generated image:
import matplotlib.pyplot as plt
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()